# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122489e+02     1.796730e+00
 * time: 0.3996889591217041
     1     1.054549e+01     1.005811e+00
 * time: 1.0921859741210938
     2    -1.091818e+01     1.220121e+00
 * time: 1.1774349212646484
     3    -3.390030e+01     9.029656e-01
 * time: 1.3191211223602295
     4    -4.726869e+01     6.927457e-01
 * time: 1.4120769500732422
     5    -5.667603e+01     2.774442e-01
 * time: 1.5158281326293945
     6    -5.970028e+01     2.160915e-01
 * time: 1.6037399768829346
     7    -6.090226e+01     7.111077e-02
 * time: 1.7000811100006104
     8    -6.137247e+01     4.373541e-02
 * time: 1.7715129852294922
     9    -6.162064e+01     5.611429e-02
 * time: 1.8468010425567627
    10    -6.182676e+01     3.082099e-02
 * time: 1.9204070568084717
    11    -6.198762e+01     2.292018e-02
 * time: 1.9950950145721436
    12    -6.208280e+01     1.757903e-02
 * time: 2.0813000202178955
    13    -6.214021e+01     1.321318e-02
 * time: 2.1488120555

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671062
    AtomicLocal         -18.8557662
    AtomicNonlocal      14.8522640
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485363 
    Xc                  -19.3336817

    total               -62.261666461444